In [1]:
import pandas as pd

df=pd.read_csv(r"E:\Github Projects\Stock Market\stocks\JPM.csv")
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,1980-03-17,0.0,5.129630,5.018518,5.037037,0.293599,62400
1,1980-03-18,0.0,5.111111,5.037037,5.074074,0.295758,63900
2,1980-03-19,0.0,5.166667,5.111111,5.148148,0.300075,40500
3,1980-03-20,0.0,5.148148,5.092593,5.111111,0.297916,18900
4,1980-03-21,0.0,5.222222,5.111111,5.222222,0.304393,97200


In [2]:
df.dtypes

Date          object
Open         float64
High         float64
Low          float64
Close        float64
Adj Close    float64
Volume         int64
dtype: object

<span style="color: blue; font-size: 15px;">We can see that the datatype of 'Date' is object and not datetime
We can calculate the moving average and daily returns for the ETL Pipeline</span>

 <span style="color: blue; font-size: 15px;">ETL Pipeline Outline:</span><br> 
 <span style="color: blue; font-size: 15px;">-- Extract the dataset for Stocks</span>  
 <span style="color: blue; font-size: 15px;">-- Transform the 'Date' column to datetime from object and calculate Moving</span><span style="color: blue; font-size: 15px;">Averages and Daily Return for each Stock dataset</span>      
<span style="color: blue; font-size: 15px;">-- Load the transformed data to table to SSMS</span>

In [ ]:
import pyodbc
df = pd.read_csv(r"E:\Github Projects\Stock Market\stocks\JPM.csv")

# Transform
df['Date'] = pd.to_datetime(df['Date'])
df['30_day_MA'] = df['Adj Close'].rolling(window=30).mean()
df['100_day_MA'] = df['Adj Close'].rolling(window=100).mean()
df['Daily_Return'] = df['Adj Close'].pct_change().round(4)

# Load
server = 'AYUSH\SQLEXPRESS'
database = 'Stock Market'

conn_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};TRUSTED_CONNECTION=yes'
cnxn = pyodbc.connect(conn_string)
cursor = cnxn.cursor()

insert_query = '''
INSERT INTO dbo.Stocks ([Date], [Open], [High], [Low], [Close], [Adj Close], [Volume], [30_day_MA], [100_day_MA], [Daily_Return]) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
'''
df.fillna(value=pd.NA, inplace=True)
df = df.where(pd.notna(df), None)

for index, row in df.iterrows():
    cursor.execute(insert_query, 
                   row['Date'].to_pydatetime(),  # Ensure Date is a Python datetime, not pandas Timestamp
                   row['Open'],
                   row['High'],
                   row['Low'],
                   row['Close'],
                   row['Adj Close'],
                   row['Volume'],
                   row['30_day_MA'],
                   row['100_day_MA'],
                   row['Daily_Return'])
    cnxn.commit()

cursor.close()
cnxn.close()